# PyTorch Example

## Install Dependencies

In [ ]:
#@title Install and Import Dependencies

# this assumes that you have a relevant version of PyTorch installed
!pip install -q torchaudio soundfile

import glob
import torch
torch.set_num_threads(1)

from IPython.display import Audio
from pprint import pprint

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)

(get_speech_ts,
 save_audio,
 read_audio,
 state_generator,
 single_audio_stream,
 collect_speeches) = utils

files_dir = torch.hub.get_dir() + '/snakers4_silero-vad_master/files'

## Full Audio

In [ ]:
wav = read_audio(f'{files_dir}/en.wav')
# get speech timestamps from full audio file
speech_timestamps = get_speech_ts(wav, model,
                                  num_steps=4)
pprint(speech_timestamps)

In [ ]:
# merge all speech chunks to one audio
save_audio('only_speech.wav',
           collect_speeches(speech_timestamps, wav), 16000) 
Audio('only_speech.wav')

## Single Audio Stream

In [ ]:
wav = f'{files_dir}/en.wav'

for batch in single_audio_stream(model, wav):
    if batch:
        print(batch)

## Multiple Audio Streams

In [ ]:
audios_for_stream = glob.glob(f'{files_dir}/*.wav')
len(audios_for_stream) # total 4 audios

In [ ]:
for batch in state_generator(model, audios_for_stream, audios_in_stream=2): # 2 audio stream
    if batch:
        pprint(batch)

# ONNX Example

## Install Dependencies

In [ ]:
#@title Install and Import Dependencies

# this assumes that you have a relevant version of PyTorch installed
!pip install -q torchaudio soundfile onnxruntime

import glob
import onnxruntime
from pprint import pprint

from IPython.display import Audio

_, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)

(get_speech_ts,
 save_audio,
 read_audio,
 state_generator,
 single_audio_stream,
 collect_speeches) = utils

files_dir = torch.hub.get_dir() + '/snakers4_silero-vad_master/files'

def init_onnx_model(model_path: str):
    return onnxruntime.InferenceSession(model_path)

def validate_onnx(model, inputs):
    with torch.no_grad():
        ort_inputs = {'input': inputs.cpu().numpy()}
        outs = model.run(None, ort_inputs)
        outs = [torch.Tensor(x) for x in outs]
    return outs[0]

## Full Audio

In [ ]:
model = init_onnx_model(f'{files_dir}/model.onnx')
wav = read_audio(f'{files_dir}/en.wav')

# get speech timestamps from full audio file
speech_timestamps = get_speech_ts(wav, model, num_steps=4, run_function=validate_onnx) 
pprint(speech_timestamps)

In [ ]:
# merge all speech chunks to one audio
save_audio('only_speech.wav', collect_speeches(speech_timestamps, wav), 16000)
Audio('only_speech.wav')

## Single Audio Stream

In [ ]:
model = init_onnx_model(f'{files_dir}/model.onnx')
wav = f'{files_dir}/en.wav'

In [ ]:
for batch in single_audio_stream(model, wav, run_function=validate_onnx):
    if batch:
        pprint(batch)

## Multiple Audio Streams

In [ ]:
model = init_onnx_model(f'{files_dir}/model.onnx')
audios_for_stream = glob.glob(f'{files_dir}/*.wav')
pprint(len(audios_for_stream)) # total 4 audios

In [ ]:
for batch in state_generator(model, audios_for_stream, audios_in_stream=2, run_function=validate_onnx): # 2 audio stream
    if batch:
        pprint(batch)